In [1]:
%matplotlib QT5
import numpy as np
import matplotlib.pyplot as plt

import pyaccel
import pymodels
import idanalysis.optics as opt
from apsuite import lattice_errors
from apsuite.orbcorr import OrbitCorr, CorrParams
from mathphys.functions import save_pickle, load_pickle


Load data

In [10]:
data_mach = load_pickle('20_machines_seed_4736_sext_ramp.pickle')


In [11]:
params = data_mach['orbcorr_params']
print(params.tolerance)
print(params.minsingval)
print(params.maxnriters)


1e-09
0.5
12


In [4]:
def create_id():
    ids = list()
    IDModel = pymodels.si.IDModel
    papu50 = IDModel(
        subsec=IDModel.SUBSECTIONS.ID17SA,
        file_name='/home/gabriel/repos/idanalysis/scripts/papu50/results/model/kickmaps/kickmap-ID_width40_phasepos00p000_gap24p0-shifted_on_axis.txt',
        fam_name='PAPU50', nr_steps=40,
        rescale_kicks=10, rescale_length=1)
    ids.append(papu50)
    return ids


In [17]:
ids = create_id()
ids[0].subsec


'ID17SA'

In [2]:
# phase, dgv
config_dict = dict()
config_dict[(0, 0)] = 0
config_dict[(0, 13.125)] = 1
config_dict[(0, 26.25)] = 2
config_dict[(-13.125, 13.125)] = 3
config_dict[(-13.125, 26.25)] = 4
config_dict[(-26.25, 13.125)] = 5
config_dict[(-26.25, 26.25)] = 6


In [3]:
data = load_pickle('/media/gabriel/System/Gabriel/DELTA52/dynapt_342080_nominal.pickle')
x0, y0 = data[0], data[1]


In [4]:
x = np.zeros((len(config_dict.values()), x0.shape[0], x0.shape[1]))
y = np.zeros((len(config_dict.values()), x0.shape[0], x0.shape[1]))
print(x.shape)
for config, nr in config_dict.items():
    fname = '/media/gabriel/System/Gabriel/DELTA52/dynapt_342080_config{}.pickle'.format(nr)
    data = load_pickle(fname)
    x[nr, :, :] = data[0]
    y[nr, :, :] = data[1]


(7, 20, 9)


In [5]:
xm = np.zeros((len(config_dict.values()), x0.shape[0], x0.shape[1]))
ym = np.zeros((len(config_dict.values()), x0.shape[0], x0.shape[1]))
print(xm.shape)
for config, nr in config_dict.items():
    fname = '/media/gabriel/System/Gabriel/DELTA52/meas_dynapt_342080_config{}.pickle'.format(nr)
    data = load_pickle(fname)
    xm[nr, :, :] = data[0]
    ym[nr, :, :] = data[1]


(7, 20, 9)


In [8]:
x_mean = np.mean(x0, 0)
x_rms = np.std(x0, 0)
y_mean = np.mean(y0, 0)
y_rms = np.std(y0, 0)

x_mean1 = np.mean(x, 1)
x_rms1 = np.std(x, 1)
y_mean1 = np.mean(y, 1)
y_rms1 = np.std(y, 1)

x_mean_m = np.mean(xm, 1)
x_rms_m = np.std(xm, 1)
y_mean_m = np.mean(ym, 1)
y_rms_m = np.std(ym, 1)


fig, axs = plt.subplots(2, 1, sharex=True)
fig.suptitle('DA for 20 machines w/ random errors')
fig.set_figwidth(8)
fig.set_figheight(6)
for i in np.arange(x_mean1.shape[0]):
    if i ==0:
        axs[0].plot(1e3*x_mean1[i, :], 1e3*y_mean1[i, :], '-', label='W/ Delta', color='C1', alpha=.8)
        axs[1].plot(1e3*x_mean_m[i, :], 1e3*y_mean_m[i, :], '-', label='W/ Delta', color='C0', alpha=.8)
    else:
        axs[0].plot(1e3*x_mean1[i, :], 1e3*y_mean1[i, :], '-', color='C1', alpha=.8)
        axs[1].plot(1e3*x_mean_m[i, :], 1e3*y_mean_m[i, :], '-', color='C0', alpha=.8)
    axs[0].plot(1e3*(x_mean1[i, :]-x_rms1[i, :]), 1e3*(y_mean1[i, :]+y_rms1[i, :]), '--', color='C1', alpha=.2)
    axs[0].plot(1e3*(x_mean1[i, :]+x_rms1[i, :]), 1e3*(y_mean1[i, :]-y_rms1[i, :]), '--', color='C1', alpha=.2)
    axs[1].plot(1e3*(x_mean_m[i, :]-x_rms_m[i, :]), 1e3*(y_mean_m[i, :]+y_rms1[i, :]), '--', color='C0', alpha=.2)
    axs[1].plot(1e3*(x_mean_m[i, :]+x_rms_m[i, :]), 1e3*(y_mean_m[i, :]-y_rms1[i, :]), '--', color='C0', alpha=.2)
axs[0].plot(1e3*x_mean, 1e3*y_mean, '.-', color='k', label='Nominal')
axs[0].plot(1e3*(x_mean-x_rms), 1e3*(y_mean+y_rms), '--', color='k')
axs[0].plot(1e3*(x_mean+x_rms), 1e3*(y_mean-y_rms), '--', color='k')
axs[1].plot(1e3*x_mean, 1e3*y_mean, '.-', color='k', label='Nominal')
axs[1].plot(1e3*(x_mean-x_rms), 1e3*(y_mean+y_rms), '--', color='k')
axs[1].plot(1e3*(x_mean+x_rms), 1e3*(y_mean-y_rms), '--', color='k')
axs[1].set_xlabel('x [mm]')
axs[0].set_ylabel('y [mm]')
axs[0].legend()
axs[1].legend()
axs[0].grid()
axs[1].grid()
axs[0].set_title('Fitted model')
axs[1].set_title('Fieldmap measurements')
plt.savefig('dynapt_delta.png')


In [6]:
x_mean = np.mean(x0, 0)
x_rms = np.std(x0, 0)
y_mean = np.mean(y0, 0)
y_rms = np.std(y0, 0)

x_mean1 = np.mean(x, 1)
x_rms1 = np.std(x, 1)
y_mean1 = np.mean(y, 1)
y_rms1 = np.std(y, 1)

x_mean_m = np.mean(xm, 1)
x_rms_m = np.std(xm, 1)
y_mean_m = np.mean(ym, 1)
y_rms_m = np.std(ym, 1)


fig, axs = plt.subplots(1, 1, sharex=True)
fig.suptitle('DA for 20 machines w/ random errors')
fig.set_figwidth(8)
fig.set_figheight(6)
for i in np.arange(x_mean1.shape[0]):
    if i ==0:
        axs.plot(1e3*x_mean_m[i, :], 1e3*y_mean_m[i, :], '-', label='W/ Delta', color='C0', alpha=.8)
    else:

        axs.plot(1e3*x_mean_m[i, :], 1e3*y_mean_m[i, :], '-', color='C0', alpha=.8)

    axs.plot(1e3*(x_mean_m[i, :]-x_rms_m[i, :]), 1e3*(y_mean_m[i, :]+y_rms1[i, :]), '--', color='C0', alpha=.2)
    axs.plot(1e3*(x_mean_m[i, :]+x_rms_m[i, :]), 1e3*(y_mean_m[i, :]-y_rms1[i, :]), '--', color='C0', alpha=.2)

axs.plot(1e3*x_mean, 1e3*y_mean, '.-', color='k', label='Nominal')
axs.plot(1e3*(x_mean-x_rms), 1e3*(y_mean+y_rms), '--', color='k')
axs.plot(1e3*(x_mean+x_rms), 1e3*(y_mean-y_rms), '--', color='k')
axs.set_xlabel('x [mm]')

axs.legend()

axs.grid()

axs.set_title('Fieldmap measurements')
plt.savefig('dynapt_delta_meas.png')
